In [ ]:
pip install ucimlrepo

In [9]:
from ucimlrepo import fetch_ucirepo
car_evaluation = fetch_ucirepo(id=19)

X = car_evaluation.data.features
Y = car_evaluation.data.targets
print(X)
print ("--------------")
print(Y)
print ("--------------")


     buying  maint  doors persons lug_boot safety
0     vhigh  vhigh      2       2    small    low
1     vhigh  vhigh      2       2    small    med
2     vhigh  vhigh      2       2    small   high
3     vhigh  vhigh      2       2      med    low
4     vhigh  vhigh      2       2      med    med
...     ...    ...    ...     ...      ...    ...
1723    low    low  5more    more      med    med
1724    low    low  5more    more      med   high
1725    low    low  5more    more      big    low
1726    low    low  5more    more      big    med
1727    low    low  5more    more      big   high

[1728 rows x 6 columns]
--------------
      class
0     unacc
1     unacc
2     unacc
3     unacc
4     unacc
...     ...
1723   good
1724  vgood
1725  unacc
1726   good
1727  vgood

[1728 rows x 1 columns]
--------------


In [8]:
variables_df = car_evaluation.variables
print (variables_df)

       name     role  ... units missing_values
0    buying  Feature  ...  None             no
1     maint  Feature  ...  None             no
2     doors  Feature  ...  None             no
3   persons  Feature  ...  None             no
4  lug_boot  Feature  ...  None             no
5    safety  Feature  ...  None             no
6     class   Target  ...  None             no

[7 rows x 7 columns]


In [10]:
import pandas as pd
combined_df= pd.concat([X, Y], axis = 1)
with pd.ExcelWriter('car_evaluation.xlsx', engine= 'openpyxl') as writer:
    combined_df.to_excel(writer, sheet_name="Variables Metadata", index=False)

In [ ]:
# from sklearn.model_selection import train_test_split #Splits your dataset into training and testing sets.
# from sklearn.ensemble import RandomForestClassifier #"forest" of decision trees and combines their outputs for classification tasks.
# from sklearn.metrics import classification_report, confusion_matrix #classification_report: Gives precision, recall, f1-score, and accuracy for each class. confusion_matrix: Shows the number of correct/incorrect predictions for each class in a matrix form.
# from sklearn.preprocessing import StandardScaler #Imports a tool to standardize features by removing the mean and scaling to unit variance.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler


x = combined_df.drop(columns=["class"])
y = combined_df["class"]

scalar = StandardScaler()
x_scaled = scaler.fit_transform(x)



KeyError: 'quality'